In [4]:
import lightgbm as lgb
import pandas as pd
import numpy as np
from sklearn.preprocessing import *
import matplotlib.pyplot as plt
from datetime import datetime
import gc
import xgboost as xgb

In [2]:
train_df = pd.read_csv('./data/feature/train.csv',parse_dates=['date','time_start'],dtype={'link_ID':str})
train_df = train_df.sort_values(by=['link_ID','month','day','time_start_hour'])

In [3]:
grp = train_df.groupby(['link_ID','month','day','time_start_hour'])

In [ ]:
k = list()
for i,g in grp:
    m = g['travel_time'].mean()
    s = g['travel_time'].std()
    k.append(g['travel_time'].clip_upper(m+2.5*s))

In [5]:
train_df['travel_time'].mean()

15.440440186781608

In [6]:
v = pd.concat(k)

In [7]:
train_df['travel_time'] = v

In [9]:
train_df.to_csv('./data/feature/train.csv',index=False)

In [ ]:
train_df = train_df[train_df.date>=pd.to_datetime('20160302',format='%Y%m%d')]
train_df.fillna(method='ffill',inplace=True)
val_data = train_df[(train_df.time_start_hour==8)&(train_df.month==5)]
val_index = val_data.index
test_data = train_df[(train_df.month==6)&(train_df.time_start_hour==8)]
test_index = test_data.index
rm_index = val_index.append(test_index)
rm_index = train_df.index.isin(rm_index)
train_data = train_df[~rm_index]
train_data = train_data[train_df.time_start<pd.Timestamp('2016-07-01-00')]
test_id = test_data.link_ID.values
test_time = test_data.time_start.values

In [ ]:
del train_df
gc.collect()

In [ ]:
col_list = train_data.drop(['link_ID','date','time_start','travel_time'],axis=1).columns.tolist()
train_y = train_data.travel_time.values
val_y = val_data.travel_time.values
train_data = train_data.drop(['link_ID','date','time_start','travel_time'],axis=1).values
val_data = val_data.drop(['link_ID','date','time_start','travel_time'],axis=1).values
dtest = test_data.drop(['link_ID','date','time_start','travel_time'],axis=1).values

In [ ]:
dtrain = lgb.Dataset(train_data,train_y,feature_name=col_list,categorical_feature=['link_ID','class'])
dval = lgb.Dataset(val_data,val_y,feature_name=col_list,categorical_feature=['link_ID','class'])
dtrain_all = lgb.Dataset(np.vstack([train_data,val_data]),np.hstack([train_y,val_y]),feature_name=col_list,categorical_feature=['link_ID','class'])

In [ ]:
def mape_eval(pred,dtrain):
    label = dtrain.get_label()
    error = (np.abs(pred-label)/label).mean()
    return 'mape',error,False

In [ ]:
param_gbdt = {'num_leaves':75, 'num_trees':1200, 'objective':'regression_l1','learning_rate':0.005,'num_threads':-1,
        'feature_fraction':0.75,'bagging_fraction':0.75,'bagging_freq':1,'max_bin':20,'gaussian_eta':0.7,
        'lambda_l1':10,'boosting':'gbdt','device':'cpu'}
#gbdt = lgb.train(param_gbdt, dtrain, feval = mape_eval,valid_sets=[dval],verbose_eval=True,num_boost_round=2500,early_stopping_rounds=20)